In [93]:
from github import Github, GithubException, BadCredentialsException
import json
import time

In [94]:
user = Github('username', 'password')
json_repos = json.loads(json.dumps({'Repositories': ()}), encoding='utf-8')
FILE_NAME = './data/repos.json'

In [76]:
def new_search_start():
    return user.get_rate_limit().search.reset

def check_remaining_rate(search_start, limit=10):
    rem = user.get_rate_limit().search.remaining
    if (rem < limit):
        to_wait = 60 - (new_search_start() - search_start).total_seconds()
        print('\tRemaining rate {0}<10 => Sleep {1} seconds'.format(rem, to_wait))
        time.sleep(to_wait)
        return new_search_start()

In [87]:
def write_into_file(repos):
    search_start = new_search_start()
    for repo in repos:
        search_start = check_remaining_rate(search_start) or search_start
        json_repos['Repositories'].append({
            'name' : repo.name,
            'language' : repo.language,
            'created_at' : str(repo.created_at),
            'forks' : repo.forks,
            'size' : repo.size,
            'watchers' : repo.watchers,
            'url' : repo.url,
        })

In [92]:
def search(query, sort, order):
    repos = user.search_repositories(query,sort,order,created='>=2018-01-01')
    write_into_file(repos)

In [ ]:
if __name__ == '__main__':
    try:
        start = time.time()
        search('gameoflife', 'stars', 'desc')
        print('\nSearch finished: {} seconds'.format(round(time.time() - start, 4)))
    except BadCredentialsException as e:
        print(e)
    except GithubException as e:
        print(e.message)
    finally:
        with open(FILE_NAME, 'w+', encoding='utf-8') as f:
            json.dump(json_repos, f, sort_keys=True, indent=2, separators=(',', ': '))
            print('\nResults written into %s !' % FILE_NAME)